In [51]:
import face_recognition
import os
import pandas as pd
import numpy as np
from glob import glob
from sklearn.ensemble import RandomForestClassifier
from scikit_checkpoint import ScikitCheckpoint
from sklearn.cluster import KMeans
np.random.seed(4)

In [52]:
data = []
target = []
face_names = []
num_faces = len(face_names)

In [53]:
faces_selected = set()
for file_path in glob(os.environ.get("DATA_DIR")+"/*/*"):
    class_name = file_path.split('/')[-2]
    if not class_name == 'test':
        if class_name not in faces_selected:
            faces_selected.add(class_name)
            face_names.append(class_name)
        load_image = face_recognition.load_image_file(file_path)
        list_encoding = face_recognition.face_encodings(load_image)
        if len(list_encoding) > 0:
            face_encoding = list_encoding[0]
            data.append(face_encoding)
            target.append(face_names.index(class_name))

In [54]:
face_names = np.array(face_names)
data = np.asarray(data)
dimensions = range(len(face_encoding))

In [60]:
df = pd.DataFrame(data, columns=dimensions)
df['is_train'] = np.random.uniform(0, 1, len(df)) <= .60
df['face'] = pd.Categorical.from_codes(target, face_names)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,120,121,122,123,124,125,126,127,is_train,face
0,-0.099646,0.150337,-0.031157,-0.067576,-0.099756,0.044092,-0.003369,-0.007982,0.147150,-0.025015,...,0.031742,0.036646,-0.160967,-0.126963,0.033340,0.011252,-0.018887,0.029731,True,mike_pence
1,-0.062825,0.100133,0.015886,-0.081562,-0.132708,0.050036,-0.068835,-0.010881,0.162145,-0.036087,...,0.008829,0.005791,-0.147862,-0.116633,0.012758,0.015249,-0.068377,0.041533,True,mike_pence
2,-0.050357,0.149516,-0.000407,-0.043841,-0.131729,0.040053,-0.025665,-0.040291,0.154646,-0.008897,...,-0.004118,-0.007885,-0.096182,-0.141149,0.031486,0.039475,-0.063956,0.014930,False,mike_pence
3,-0.108595,0.091008,0.009532,-0.097698,-0.085716,0.102835,-0.039362,-0.089313,0.130574,-0.050163,...,-0.023262,0.020501,-0.156357,-0.135478,0.013596,0.059133,-0.032501,0.041920,False,mike_pence
4,-0.126487,0.130311,0.011764,-0.039368,-0.119886,0.092979,-0.030116,-0.093928,0.104042,-0.050722,...,0.007355,0.014003,-0.156209,-0.157914,0.036138,0.007237,-0.064688,0.065969,True,mike_pence


In [61]:
train, test = df[df['is_train']==True], df[df['is_train']==False]

In [62]:
features = df.columns[:len(face_encoding)]
clf = RandomForestClassifier(n_jobs=4)
y, _ = pd.factorize(train['face'])
clf.fit(train[features], y)
checkpoint = ScikitCheckpoint(os.environ['SNAPSHOTS_DIR'], )
stats = {'label': 'random_forest'}

In [67]:
checkpoint.save_model(clf, stats)
preds = face_names[np.array(clf.predict(test[features]))]
cross_validation = pd.crosstab(test['face'], preds, rownames=['actual'], colnames=['preds'])
print(cross_validation)

preds         donald_trump  mike_pence  putin
actual                                       
mike_pence               0           6      0
donald_trump             5           0      0
putin                    0           0      5
